In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

from collections import OrderedDict
from random import randint
import random
import decimal
from operator import add
from operator import itemgetter

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
max_hidden_layers = 9
max_neurons_per_layer = 100
min_learning_rate = .0001
max_learning_rate = .01
acti_func = ["softmax", "elu", "selu", "softplus", "softsign", 
             "relu", "tanh", "sigmoid", "hard_sigmoid", "linear"]
num_individuals = 16
loss_weight = 10
accuracy_weight = 10
time_weight = 10

In [3]:
def load_data():
    iris_data = load_iris()
    print('Example Data:')
    print(iris_data.data[:5])
    print('Example Labels:')
    print(iris_data.target[:5])
    global iris_data

<ipython-input-3-11c664384486>:7: SyntaxWarning: name 'iris_data' is assigned to before global declaration
  global iris_data


In [4]:
def process_data():
    x = iris_data.data
    y_ = iris_data.target.reshape(-1,1)

    encoder = OneHotEncoder(sparse=False)
    y = encoder.fit_transform(y_)
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=.20)
    global train_x, test_x, train_y, test_y

<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'train_x' is assigned to before global declaration
  global train_x, test_x, train_y, test_y
<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'test_x' is assigned to before global declaration
  global train_x, test_x, train_y, test_y
<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'train_y' is assigned to before global declaration
  global train_x, test_x, train_y, test_y
<ipython-input-4-1082ec30e327>:8: SyntaxWarning: name 'test_y' is assigned to before global declaration
  global train_x, test_x, train_y, test_y


In [5]:
load_data()
process_data()

Example Data:
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
Example Labels:
[0 0 0 0 0]


In [6]:
def create_network(model_num):
    model_name = "Model"+str(model_num)
    model_desc = {'name':model_name}
    print(model_name)
    model_name = Sequential()
    num_hidden_layers = randint(0, max_hidden_layers)
    i = 0
    model_name.add(Dense(10, input_shape=(4,), activation='relu', name='input'))
    while i < num_hidden_layers:
        name = 'hidden'+str(i)
        num_neurons = randint(1, max_neurons_per_layer)
        activation = acti_func[randint(0, len(acti_func)-1)]
        model_desc.update({'hidden'+str(i) : (num_neurons, activation)})
        #print(activation)
        model_name.add(Dense(num_neurons, activation=activation, name=name))
        i = i+1
    model_name.add(Dense(3, activation='softmax', name='output'))
    global model_name
    global model_desc

<ipython-input-6-0e67b9f64f4e>:18: SyntaxWarning: name 'model_name' is assigned to before global declaration
  global model_name
<ipython-input-6-0e67b9f64f4e>:19: SyntaxWarning: name 'model_desc' is assigned to before global declaration
  global model_desc


In [7]:
def lr_and_optimizer(model_name):
    learning_rate = random.uniform(min_learning_rate, max_learning_rate)
    #print(learning_rate)
    model_desc.update({'learning_rate' : learning_rate})
    optimizer = Adam(lr=learning_rate)
    model_desc.update({'optimizer' : 'Adam'})
    model_name.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
def print_summary(model_name):
    print('Neural Net Model Summary: ')
    print(model_name.summary())

In [9]:
def train_and_test(model_name):
    model_name.fit(train_x, train_y, verbose = 2, batch_size = 5, epochs = 200)
    results = model_name.evaluate(test_x, test_y)
    global results

<ipython-input-9-915b75c55b69>:4: SyntaxWarning: name 'results' is assigned to before global declaration
  global results


In [10]:
def individual(model_num):
    create_network(model_num)
    #print_summary(model_name)
    lr_and_optimizer(model_name)
    train_and_test(model_name)
    loss_fitness =  (1/float(results[0]))*loss_weight
    acc_fitness = float(results[1])*accuracy_weight
    total_fitness = loss_fitness + acc_fitness
    model_desc.update({'loss_fitness':loss_fitness})
    model_desc.update({'accuracy_fitness':acc_fitness})
    model_desc.update({'total_fitness':total_fitness})
    print(model_name)
    print(model_desc)

In [11]:
def population():
    population_desc = []
    for x in xrange(num_individuals):
        individual(model_num = x)
        population_desc.append(model_desc)
    print(population_desc)
    global population_desc

<ipython-input-11-662326261e74>:7: SyntaxWarning: name 'population_desc' is assigned to before global declaration
  global population_desc


In [12]:
population()

Model0
Epoch 1/200
 - 0s - loss: 1.0254 - acc: 0.4833
Epoch 2/200
 - 0s - loss: 0.5833 - acc: 0.6333
Epoch 3/200
 - 0s - loss: 0.4065 - acc: 0.8250
Epoch 4/200
 - 0s - loss: 0.2074 - acc: 0.9500
Epoch 5/200
 - 0s - loss: 0.1482 - acc: 0.9417
Epoch 6/200
 - 0s - loss: 0.1084 - acc: 0.9750
Epoch 7/200
 - 0s - loss: 0.1837 - acc: 0.9333
Epoch 8/200
 - 0s - loss: 0.1071 - acc: 0.9667
Epoch 9/200
 - 0s - loss: 0.1338 - acc: 0.9667
Epoch 10/200
 - 0s - loss: 0.1451 - acc: 0.9333
Epoch 11/200
 - 0s - loss: 0.0911 - acc: 0.9667
Epoch 12/200
 - 0s - loss: 0.1140 - acc: 0.9417
Epoch 13/200
 - 0s - loss: 0.1117 - acc: 0.9750
Epoch 14/200
 - 0s - loss: 0.0984 - acc: 0.9750
Epoch 15/200
 - 0s - loss: 0.1459 - acc: 0.9333
Epoch 16/200
 - 0s - loss: 0.1163 - acc: 0.9667
Epoch 17/200
 - 0s - loss: 0.0806 - acc: 0.9833
Epoch 18/200
 - 0s - loss: 0.1092 - acc: 0.9417
Epoch 19/200
 - 0s - loss: 0.1368 - acc: 0.9417
Epoch 20/200
 - 0s - loss: 0.1097 - acc: 0.9667
Epoch 21/200
 - 0s - loss: 0.0813 - acc: 0

 - 0s - loss: 0.0599 - acc: 0.9750
Epoch 171/200
 - 0s - loss: 0.0570 - acc: 0.9833
Epoch 172/200
 - 0s - loss: 0.0525 - acc: 0.9833
Epoch 173/200
 - 0s - loss: 0.0441 - acc: 0.9917
Epoch 174/200
 - 0s - loss: 0.0456 - acc: 0.9917
Epoch 175/200
 - 0s - loss: 0.1151 - acc: 0.9583
Epoch 176/200
 - 0s - loss: 0.0963 - acc: 0.9667
Epoch 177/200
 - 0s - loss: 0.0709 - acc: 0.9833
Epoch 178/200
 - 0s - loss: 0.0750 - acc: 0.9750
Epoch 179/200
 - 0s - loss: 0.0960 - acc: 0.9583
Epoch 180/200
 - 0s - loss: 0.0842 - acc: 0.9750
Epoch 181/200
 - 0s - loss: 0.0773 - acc: 0.9750
Epoch 182/200
 - 0s - loss: 0.0605 - acc: 0.9833
Epoch 183/200
 - 0s - loss: 0.0603 - acc: 0.9750
Epoch 184/200
 - 0s - loss: 0.0619 - acc: 0.9833
Epoch 185/200
 - 0s - loss: 0.0661 - acc: 0.9750
Epoch 186/200
 - 0s - loss: 0.0989 - acc: 0.9667
Epoch 187/200
 - 0s - loss: 0.1121 - acc: 0.9500
Epoch 188/200
 - 0s - loss: 0.0921 - acc: 0.9667
Epoch 189/200
 - 0s - loss: 0.0768 - acc: 0.9750
Epoch 190/200
 - 0s - loss: 0.0565

Epoch 132/200
 - 0s - loss: 0.5283 - acc: 0.6083
Epoch 133/200
 - 0s - loss: 0.5124 - acc: 0.6833
Epoch 134/200
 - 0s - loss: 0.5052 - acc: 0.6750
Epoch 135/200
 - 0s - loss: 0.5239 - acc: 0.6583
Epoch 136/200
 - 0s - loss: 0.6008 - acc: 0.5917
Epoch 137/200
 - 0s - loss: 0.5091 - acc: 0.6750
Epoch 138/200
 - 0s - loss: 0.4801 - acc: 0.7083
Epoch 139/200
 - 0s - loss: 0.5211 - acc: 0.6250
Epoch 140/200
 - 0s - loss: 0.5191 - acc: 0.5917
Epoch 141/200
 - 0s - loss: 0.5393 - acc: 0.6500
Epoch 142/200
 - 0s - loss: 0.5357 - acc: 0.6417
Epoch 143/200
 - 0s - loss: 0.4981 - acc: 0.6583
Epoch 144/200
 - 0s - loss: 0.4934 - acc: 0.6750
Epoch 145/200
 - 0s - loss: 0.5365 - acc: 0.6250
Epoch 146/200
 - 0s - loss: 0.5021 - acc: 0.5667
Epoch 147/200
 - 0s - loss: 0.5094 - acc: 0.6500
Epoch 148/200
 - 0s - loss: 0.5018 - acc: 0.6333
Epoch 149/200
 - 0s - loss: 0.5035 - acc: 0.6583
Epoch 150/200
 - 0s - loss: 0.5172 - acc: 0.6167
Epoch 151/200
 - 0s - loss: 0.5084 - acc: 0.6000
Epoch 152/200
 - 0s 

Epoch 92/200
 - 0s - loss: 0.0623 - acc: 0.9750
Epoch 93/200
 - 0s - loss: 0.1759 - acc: 0.9500
Epoch 94/200
 - 0s - loss: 0.1369 - acc: 0.9250
Epoch 95/200
 - 0s - loss: 0.1385 - acc: 0.9333
Epoch 96/200
 - 0s - loss: 0.0694 - acc: 0.9833
Epoch 97/200
 - 0s - loss: 0.0759 - acc: 0.9667
Epoch 98/200
 - 0s - loss: 0.0701 - acc: 0.9750
Epoch 99/200
 - 0s - loss: 0.0682 - acc: 0.9667
Epoch 100/200
 - 0s - loss: 0.0586 - acc: 0.9667
Epoch 101/200
 - 0s - loss: 0.1204 - acc: 0.9333
Epoch 102/200
 - 0s - loss: 0.1138 - acc: 0.9583
Epoch 103/200
 - 0s - loss: 0.0698 - acc: 0.9667
Epoch 104/200
 - 0s - loss: 0.0785 - acc: 0.9500
Epoch 105/200
 - 0s - loss: 0.1135 - acc: 0.9333
Epoch 106/200
 - 0s - loss: 0.0922 - acc: 0.9500
Epoch 107/200
 - 0s - loss: 0.1826 - acc: 0.9333
Epoch 108/200
 - 0s - loss: 0.0787 - acc: 0.9750
Epoch 109/200
 - 0s - loss: 0.0631 - acc: 0.9583
Epoch 110/200
 - 0s - loss: 0.0938 - acc: 0.9500
Epoch 111/200
 - 0s - loss: 0.0652 - acc: 0.9667
Epoch 112/200
 - 0s - loss: 

 - 0s - loss: 0.4977 - acc: 0.5917
Epoch 52/200
 - 0s - loss: 0.4969 - acc: 0.6000
Epoch 53/200
 - 0s - loss: 0.4968 - acc: 0.6500
Epoch 54/200
 - 0s - loss: 0.4973 - acc: 0.6500
Epoch 55/200
 - 0s - loss: 0.4965 - acc: 0.6000
Epoch 56/200
 - 0s - loss: 0.4961 - acc: 0.6250
Epoch 57/200
 - 0s - loss: 0.4968 - acc: 0.6083
Epoch 58/200
 - 0s - loss: 0.4959 - acc: 0.6500
Epoch 59/200
 - 0s - loss: 0.4961 - acc: 0.5833
Epoch 60/200
 - 0s - loss: 0.4954 - acc: 0.6500
Epoch 61/200
 - 0s - loss: 0.4957 - acc: 0.6500
Epoch 62/200
 - 0s - loss: 0.4970 - acc: 0.6500
Epoch 63/200
 - 0s - loss: 0.4950 - acc: 0.6500
Epoch 64/200
 - 0s - loss: 0.4966 - acc: 0.5917
Epoch 65/200
 - 0s - loss: 0.4956 - acc: 0.6083
Epoch 66/200
 - 0s - loss: 0.4949 - acc: 0.6500
Epoch 67/200
 - 0s - loss: 0.4954 - acc: 0.6500
Epoch 68/200
 - 0s - loss: 0.4956 - acc: 0.5750
Epoch 69/200
 - 0s - loss: 0.4948 - acc: 0.6333
Epoch 70/200
 - 0s - loss: 0.4950 - acc: 0.6500
Epoch 71/200
 - 0s - loss: 0.4943 - acc: 0.6500
Epoch

...

Epoch 30/200
 - 0s - loss: 0.5656 - acc: 0.6500
Epoch 31/200
 - 0s - loss: 0.5630 - acc: 0.6500
Epoch 32/200
 - 0s - loss: 0.5604 - acc: 0.6500
Epoch 33/200
 - 0s - loss: 0.5577 - acc: 0.6500
Epoch 34/200
 - 0s - loss: 0.5551 - acc: 0.6500
Epoch 35/200
 - 0s - loss: 0.5529 - acc: 0.6500
Epoch 36/200
 - 0s - loss: 0.5505 - acc: 0.6500
Epoch 37/200
 - 0s - loss: 0.5484 - acc: 0.6500
Epoch 38/200
 - 0s - loss: 0.5467 - acc: 0.6500
Epoch 39/200
 - 0s - loss: 0.5447 - acc: 0.6500
Epoch 40/200
 - 0s - loss: 0.5431 - acc: 0.6500
Epoch 41/200
 - 0s - loss: 0.5411 - acc: 0.6500
Epoch 42/200
 - 0s - loss: 0.5396 - acc: 0.6500
Epoch 43/200
 - 0s - loss: 0.5388 - acc: 0.6500
Epoch 44/200
 - 0s - loss: 0.5365 - acc: 0.6500
Epoch 45/200
 - 0s - loss: 0.5365 - acc: 0.6500
Epoch 46/200
 - 0s - loss: 0.5337 - acc: 0.6500
Epoch 47/200
 - 0s - loss: 0.5328 - acc: 0.6500
Epoch 48/200
 - 0s - loss: 0.5314 - acc: 0.6500
Epoch 49/200
 - 0s - loss: 0.5304 - acc: 0.6500
Epoch 50/200
 - 0s - loss: 0.5291 - acc:

Epoch 199/200
 - 0s - loss: 0.1176 - acc: 0.9667
Epoch 200/200
 - 0s - loss: 0.0885 - acc: 0.9750
30/30 [==============================] - 0s 5ms/step
{'hidden2': (65, 'softmax'), 'hidden1': (14, 'elu'), 'hidden0': (86, 'hard_sigmoid'), 'optimizer': 'Adam', 'name': 'Model12', 'learning_rate': 0.0023951338253131553, 'loss_fitness': 476.9937865727637, 'total_fitness': 486.9937865727637, 'accuracy_fitness': 10.0}
Model13
Epoch 1/200
 - 0s - loss: 1.0222 - acc: 0.5333
Epoch 2/200
 - 0s - loss: 0.7388 - acc: 0.7000
Epoch 3/200
 - 0s - loss: 0.5159 - acc: 0.7917
Epoch 4/200
 - 0s - loss: 0.4125 - acc: 0.8083
Epoch 5/200
 - 0s - loss: 0.2942 - acc: 0.8500
Epoch 6/200
 - 0s - loss: 0.1900 - acc: 0.9500
Epoch 7/200
 - 0s - loss: 0.1799 - acc: 0.9500
Epoch 8/200
 - 0s - loss: 0.1700 - acc: 0.9583
Epoch 9/200
 - 0s - loss: 0.1370 - acc: 0.9583
Epoch 10/200
 - 0s - loss: 0.2244 - acc: 0.9000
Epoch 11/200
 - 0s - loss: 0.1810 - acc: 0.9167
Epoch 12/200
 - 0s - loss: 0.1298 - acc: 0.9333
Epoch 13/20

Epoch 161/200
 - 0s - loss: 0.0947 - acc: 0.9667
Epoch 162/200
 - 0s - loss: 0.0578 - acc: 0.9833
Epoch 163/200
 - 0s - loss: 0.0763 - acc: 0.9583
Epoch 164/200
 - 0s - loss: 0.0934 - acc: 0.9667
Epoch 165/200
 - 0s - loss: 0.1253 - acc: 0.9500
Epoch 166/200
 - 0s - loss: 0.1358 - acc: 0.9417
Epoch 167/200
 - 0s - loss: 0.1405 - acc: 0.9500
Epoch 168/200
 - 0s - loss: 0.0641 - acc: 0.9750
Epoch 169/200
 - 0s - loss: 0.0788 - acc: 0.9833
Epoch 170/200
 - 0s - loss: 0.0697 - acc: 0.9750
Epoch 171/200
 - 0s - loss: 0.0681 - acc: 0.9750
Epoch 172/200
 - 0s - loss: 0.0904 - acc: 0.9500
Epoch 173/200
 - 0s - loss: 0.0615 - acc: 0.9750
Epoch 174/200
 - 0s - loss: 0.0799 - acc: 0.9500
Epoch 175/200
 - 0s - loss: 0.0743 - acc: 0.9667
Epoch 176/200
 - 0s - loss: 0.0877 - acc: 0.9667
Epoch 177/200
 - 0s - loss: 0.0836 - acc: 0.9667
Epoch 178/200
 - 0s - loss: 0.0894 - acc: 0.9583
Epoch 179/200
 - 0s - loss: 0.0694 - acc: 0.9750
Epoch 180/200
 - 0s - loss: 0.0820 - acc: 0.9750
Epoch 181/200
 - 0s 

Epoch 124/200
 - 0s - loss: 0.4925 - acc: 0.6333
Epoch 125/200
 - 0s - loss: 0.4938 - acc: 0.6000
Epoch 126/200
 - 0s - loss: 0.4938 - acc: 0.5667
Epoch 127/200
 - 0s - loss: 0.4921 - acc: 0.6500
Epoch 128/200
 - 0s - loss: 0.4930 - acc: 0.6000
Epoch 129/200
 - 0s - loss: 0.4922 - acc: 0.6500
Epoch 130/200
 - 0s - loss: 0.4959 - acc: 0.5917
Epoch 131/200
 - 0s - loss: 0.4948 - acc: 0.6333
Epoch 132/200
 - 0s - loss: 0.4929 - acc: 0.6500
Epoch 133/200
 - 0s - loss: 0.4929 - acc: 0.6167
Epoch 134/200
 - 0s - loss: 0.4923 - acc: 0.6333
Epoch 135/200
 - 0s - loss: 0.4922 - acc: 0.6500
Epoch 136/200
 - 0s - loss: 0.4998 - acc: 0.6167
Epoch 137/200
 - 0s - loss: 0.4942 - acc: 0.6083
Epoch 138/200
 - 0s - loss: 0.4959 - acc: 0.6000
Epoch 139/200
 - 0s - loss: 0.4955 - acc: 0.6500
Epoch 140/200
 - 0s - loss: 0.4950 - acc: 0.6083
Epoch 141/200
 - 0s - loss: 0.4961 - acc: 0.5917
Epoch 142/200
 - 0s - loss: 0.4936 - acc: 0.6500
Epoch 143/200
 - 0s - loss: 0.4946 - acc: 0.5917
Epoch 144/200
 - 0s 

 - 0s - loss: 0.4973 - acc: 0.6250
Epoch 83/200
 - 0s - loss: 0.4977 - acc: 0.5583
Epoch 84/200
 - 0s - loss: 0.4931 - acc: 0.5917
Epoch 85/200
 - 0s - loss: 0.4952 - acc: 0.6000
Epoch 86/200
 - 0s - loss: 0.5066 - acc: 0.5917
Epoch 87/200
 - 0s - loss: 0.4971 - acc: 0.6417
Epoch 88/200
 - 0s - loss: 0.5075 - acc: 0.5667
Epoch 89/200
 - 0s - loss: 0.4976 - acc: 0.6167
Epoch 90/200
 - 0s - loss: 0.5006 - acc: 0.6333
Epoch 91/200
 - 0s - loss: 0.4935 - acc: 0.6333
Epoch 92/200
 - 0s - loss: 0.4945 - acc: 0.6167
Epoch 93/200
 - 0s - loss: 0.4928 - acc: 0.6583
Epoch 94/200
 - 0s - loss: 0.4951 - acc: 0.6500
Epoch 95/200
 - 0s - loss: 0.5115 - acc: 0.5583
Epoch 96/200
 - 0s - loss: 0.5384 - acc: 0.6500
Epoch 97/200
 - 0s - loss: 0.5105 - acc: 0.5583
Epoch 98/200
 - 0s - loss: 0.5081 - acc: 0.6000
Epoch 99/200
 - 0s - loss: 0.4935 - acc: 0.6167
Epoch 100/200
 - 0s - loss: 0.5030 - acc: 0.6500
Epoch 101/200
 - 0s - loss: 0.4948 - acc: 0.6417
Epoch 102/200
 - 0s - loss: 0.5001 - acc: 0.6250
Ep

In [13]:
def grade():    
    fitness_sum = 0
    'Find average fitness for a population.'
    for x in population_desc:
        fitness_sum = x.get("total_fitness") + fitness_sum
    avg_fitness = fitness_sum/len(population_desc)
    print(avg_fitness)
    global avg_fitness

<ipython-input-13-e0a3493f5de1>:8: SyntaxWarning: name 'avg_fitness' is assigned to before global declaration
  global avg_fitness


In [14]:
grade()

734.808732221


In [15]:
parents = []
for x in population_desc:
    if x.get("total_fitness") >= avg_fitness:
        parents.append((x.get('name'), x.get('total_fitness')))
        difference = x.get('total_fitness') - avg_fitness
        #print(x.get("name")+"    "+str(x.get("total_fitness"))+"    "+str(difference))
def get_key(item):
    return item[1]
parents_sorted = sorted(parents, key = get_key, reverse = True)

if len(parents) % 2 !=0:
    parents_sorted.pop()
    print(parents_sorted)
else:
    print(parents_sorted)

[('Model0', 3248.4254783613096), ('Model6', 2894.5059186549706), ('Model10', 1763.2754439147611), ('Model2', 1486.2712411098234)]


In [27]:
def create_child(child_num, model_1, model_2):
    parent1 = (item for item in population_desc if item["name"] == model_1).next()
    parent2 = (item for item in population_desc if item["name"] == model_2).next()
    print(parent1)
    print(parent2)
    new_learning_rate = (parent1.get("learning_rate")+parent2.get("learning_rate"))/2
    print(new_learning_rate)
    def parent_select():
        num = randint(0,1)
        if num == 0:
            return parent1
        else:
            return parent2
    model_name = "Child"+str(child_num)
    model_desc = {'name':child_name}
    print(child_name)
    model_name = Sequential()
    model_name.add(Dense(10, input_shape=(4,), activation='relu', name='input'))
    avg_hidden = 0
    parent1_hidden = 0
    parent2_hidden = 0
    for key in parent1.keys():
        if "hidden" in key:
            parent1_hidden = parent1_hidden + 1
    for key in parent2.keys():
        if "hidden" in key:
            parent2_hidden = parent2_hidden + 1
    avg_hidden = (parent1_hidden + parent2_hidden)/2
    print(avg_hidden)
    i = 0
    while i < avg_hidden:
        name = 'hidden'+str(i)
        try:
            num_neurons = parent_select().get(name)[0]
            activation = parent_select().get(name)[1]
        except:
            try:
                num_neurons = parent1.get(name)[0]
                activation = parent1.get(name)[1]
            except:
                num_neurons = parent2.get(name)[0]
                activation = parent2.get(name)[1]
        model_desc.update({'hidden'+str(i) : (num_neurons, activation)})
        #print(activation)
        model_name.add(Dense(num_neurons, activation=activation, name=name))
        i = i+1
    model_name.add(Dense(3, activation='softmax', name='output'))
    model_desc.update({'learning_rate' : new_learning_rate})
    optimizer = Adam(lr=new_learning_rate)
    model_desc.update({'optimizer' : 'Adam'})
    model_name.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
def child(child_num, model_1, model_2):
    create_child(child_num, model_1, model_2)
    print_summary(child_name)
    lr_and_optimizer(child_name)
    train_and_test(child_name)
    loss_fitness =  (1/float(results[0]))*loss_weight
    acc_fitness = float(results[1])*accuracy_weight
    total_fitness = loss_fitness + acc_fitness
    model_desc.update({'loss_fitness':loss_fitness})
    model_desc.update({'accuracy_fitness':acc_fitness})
    model_desc.update({'total_fitness':total_fitness})
    print(model_name)
    print(model_desc)

In [38]:
def child_populations():
    population_desc = []
    for x in xrange(len(parents)):
        try:
            child(x, parents_sorted[x*2][0], parents_sorted[(x*2)+1][0])
            population_desc.append(child_desc)
        except:
            pass
    print(population_desc)

In [39]:
child_populations()

{'hidden3': (83, 'elu'), 'hidden2': (33, 'softplus'), 'hidden1': (86, 'elu'), 'hidden0': (88, 'softsign'), 'optimizer': 'Adam', 'name': 'Model0', 'learning_rate': 0.0012489600449276383, 'loss_fitness': 3238.4254783613096, 'total_fitness': 3248.4254783613096, 'accuracy_fitness': 10.0}
{'hidden3': (87, 'softmax'), 'hidden2': (42, 'softplus'), 'hidden1': (16, 'sigmoid'), 'hidden0': (86, 'linear'), 'hidden7': (70, 'relu'), 'name': 'Model6', 'hidden5': (28, 'relu'), 'hidden4': (71, 'softplus'), 'learning_rate': 0.00013985712252201138, 'hidden8': (46, 'relu'), 'loss_fitness': 2884.5059186549706, 'accuracy_fitness': 10.0, 'total_fitness': 2894.5059186549706, 'optimizer': 'Adam', 'hidden6': (96, 'tanh')}
0.000694408583725
6
Neural Net Model Summary: 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Dense)                (None, 10)                50        
_________________________________________________

Epoch 134/200
 - 0s - loss: 0.1065 - acc: 0.9583
Epoch 135/200
 - 0s - loss: 0.0726 - acc: 0.9667
Epoch 136/200
 - 0s - loss: 0.0690 - acc: 0.9833
Epoch 137/200
 - 0s - loss: 0.0645 - acc: 0.9833
Epoch 138/200
 - 0s - loss: 0.0788 - acc: 0.9667
Epoch 139/200
 - 0s - loss: 0.1177 - acc: 0.9500
Epoch 140/200
 - 0s - loss: 0.0561 - acc: 0.9583
Epoch 141/200
 - 0s - loss: 0.0692 - acc: 0.9750
Epoch 142/200
 - 0s - loss: 0.0580 - acc: 0.9833
Epoch 143/200
 - 0s - loss: 0.0622 - acc: 0.9833
Epoch 144/200
 - 0s - loss: 0.0605 - acc: 0.9833
Epoch 145/200
 - 0s - loss: 0.0866 - acc: 0.9667
Epoch 146/200
 - 0s - loss: 0.0650 - acc: 0.9833
Epoch 147/200
 - 0s - loss: 0.0881 - acc: 0.9667
Epoch 148/200
 - 0s - loss: 0.4367 - acc: 0.8000
Epoch 149/200
 - 0s - loss: 0.1679 - acc: 0.9333
Epoch 150/200
 - 0s - loss: 0.1552 - acc: 0.9417
Epoch 151/200
 - 0s - loss: 0.1488 - acc: 0.9417
Epoch 152/200
 - 0s - loss: 0.1517 - acc: 0.9250
Epoch 153/200
 - 0s - loss: 0.0704 - acc: 0.9667
Epoch 154/200
 - 0s 

 - 0s - loss: 0.0543 - acc: 0.9833
Epoch 61/200
 - 0s - loss: 0.0596 - acc: 0.9750
Epoch 62/200
 - 0s - loss: 0.0551 - acc: 0.9750
Epoch 63/200
 - 0s - loss: 0.0508 - acc: 0.9833
Epoch 64/200
 - 0s - loss: 0.0506 - acc: 0.9750
Epoch 65/200
 - 0s - loss: 0.0617 - acc: 0.9833
Epoch 66/200
 - 0s - loss: 0.0607 - acc: 0.9750
Epoch 67/200
 - 0s - loss: 0.0543 - acc: 0.9750
Epoch 68/200
 - 0s - loss: 0.0530 - acc: 0.9750
Epoch 69/200
 - 0s - loss: 0.0543 - acc: 0.9750
Epoch 70/200
 - 0s - loss: 0.0519 - acc: 0.9750
Epoch 71/200
 - 0s - loss: 0.0525 - acc: 0.9833
Epoch 72/200
 - 0s - loss: 0.0523 - acc: 0.9750
Epoch 73/200
 - 0s - loss: 0.0523 - acc: 0.9750
Epoch 74/200
 - 0s - loss: 0.0606 - acc: 0.9750
Epoch 75/200
 - 0s - loss: 0.0572 - acc: 0.9833
Epoch 76/200
 - 0s - loss: 0.0537 - acc: 0.9750
Epoch 77/200
 - 0s - loss: 0.0514 - acc: 0.9750
Epoch 78/200
 - 0s - loss: 0.0541 - acc: 0.9833
Epoch 79/200
 - 0s - loss: 0.0523 - acc: 0.9833
Epoch 80/200
 - 0s - loss: 0.0562 - acc: 0.9750
Epoch

In [43]:
print(population_desc)

[{'hidden3': (83, 'elu'), 'hidden2': (33, 'softplus'), 'hidden1': (86, 'elu'), 'hidden0': (88, 'softsign'), 'optimizer': 'Adam', 'name': 'Model0', 'learning_rate': 0.0012489600449276383, 'loss_fitness': 3238.4254783613096, 'total_fitness': 3248.4254783613096, 'accuracy_fitness': 10.0}, {'hidden3': (39, 'hard_sigmoid'), 'hidden2': (14, 'selu'), 'hidden1': (94, 'elu'), 'hidden0': (77, 'sigmoid'), 'optimizer': 'Adam', 'name': 'Model1', 'hidden5': (77, 'elu'), 'hidden4': (63, 'elu'), 'learning_rate': 0.0073253442675290095, 'loss_fitness': 28.92639158298877, 'accuracy_fitness': 7.666666507720947, 'total_fitness': 36.59305809070972}, {'hidden3': (67, 'relu'), 'hidden2': (88, 'relu'), 'hidden1': (23, 'linear'), 'hidden0': (65, 'tanh'), 'optimizer': 'Adam', 'name': 'Model2', 'hidden5': (24, 'tanh'), 'hidden4': (23, 'softsign'), 'learning_rate': 0.0015698192360722246, 'loss_fitness': 1476.2712411098234, 'accuracy_fitness': 10.0, 'total_fitness': 1486.2712411098234}, {'hidden3': (87, 'elu'), 'hi

In [40]:
def new_parents():
    parents = []
    for x in population_desc:
        if x.get("total_fitness") >= avg_fitness:
            parents.append((x.get('name'), x.get('total_fitness')))
            difference = x.get('total_fitness') - avg_fitness
            #print(x.get("name")+"    "+str(x.get("total_fitness"))+"    "+str(difference))
    def get_key(item):
        return item[1]
    parents_sorted = sorted(parents, key = get_key, reverse = True)

    if len(parents) % 2 !=0:
        parents_sorted.pop()
        print(parents_sorted)
    else:
        print(parents_sorted)

In [41]:
new_parents()

[('Model15', 83886090.0), ('Model0', 3248.4254783613096), ('Model6', 2894.5059186549706), ('Model10', 1763.2754439147611), ('Model2', 1486.2712411098234), ('Model7', 743.5690045179314)]
